In [ ]:
import pandas as pd

# Tạo dữ liệu dưới dạng dictionary
data = {
    "id": [1,2,3,4,5,6,7,8,9,10,11,12,13,14],
    "outlook": ["sunny","sunny","overcast","rainy","rainy","rainy","overcast",
                "sunny","sunny","rainy","sunny","overcast","overcast","rainy"],
    "temperature": ["hot","hot","hot","mild","cool","cool","cool",
                    "mild","cool","mild","mild","mild","hot","mild"],
    "humidity": ["high","high","high","high","normal","normal","normal",
                 "high","normal","normal","normal","high","normal","high"],
    "wind": ["weak","strong","weak","weak","weak","strong","strong",
             "weak","weak","weak","strong","strong","weak","strong"],
    "play": ["no","no","yes","yes","yes","no","yes",
             "no","yes","yes","yes","yes","yes","no"]
}

# Tạo DataFrame
df = pd.DataFrame(data)

# Hiển thị bảng
print(df)


    id   outlook temperature humidity    wind play
0    1     sunny         hot     high    weak   no
1    2     sunny         hot     high  strong   no
2    3  overcast         hot     high    weak  yes
3    4     rainy        mild     high    weak  yes
4    5     rainy        cool   normal    weak  yes
5    6     rainy        cool   normal  strong   no
6    7  overcast        cool   normal  strong  yes
7    8     sunny        mild     high    weak   no
8    9     sunny        cool   normal    weak  yes
9   10     rainy        mild   normal    weak  yes
10  11     sunny        mild   normal  strong  yes
11  12  overcast        mild     high  strong  yes
12  13  overcast         hot   normal    weak  yes
13  14     rainy        mild     high  strong   no


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Tách biến mục tiêu
X = df.drop(['id', 'play'], axis=1)
y = df['play']

# Áp dụng One-Hot Encoding cho các cột phân loại
categorical_cols = ['outlook', 'temperature', 'humidity', 'wind']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_cols]))

# Lấy tên cột sau khi mã hóa
encoded_col_names = encoder.get_feature_names_out(categorical_cols)
X_encoded.columns = encoded_col_names

# Nối các cột đã mã hóa với các cột không phân loại (nếu có)
# Trong trường hợp này, tất cả các cột đều là phân loại sau khi loại bỏ 'id'
# X_processed = pd.concat([X_numeric, X_encoded], axis=1) # Nếu có cột số

X_processed = X_encoded # Vì tất cả các cột đều đã được mã hóa

# Hiển thị 5 hàng đầu tiên của dữ liệu đã tiền xử lý
print("Dữ liệu đã tiền xử lý (5 hàng đầu):")
print(X_processed.head())

# Hiển thị biến mục tiêu
print("\nBiến mục tiêu:")
print(y.head())

Dữ liệu đã tiền xử lý (5 hàng đầu):
   outlook_overcast  outlook_rainy  outlook_sunny  temperature_cool  \
0               0.0            0.0            1.0               0.0   
1               0.0            0.0            1.0               0.0   
2               1.0            0.0            0.0               0.0   
3               0.0            1.0            0.0               0.0   
4               0.0            1.0            0.0               1.0   

   temperature_hot  temperature_mild  humidity_high  humidity_normal  \
0              1.0               0.0            1.0              0.0   
1              1.0               0.0            1.0              0.0   
2              1.0               0.0            1.0              0.0   
3              0.0               1.0            1.0              0.0   
4              0.0               0.0            0.0              1.0   

   wind_strong  wind_weak  
0          0.0        1.0  
1          1.0        0.0  
2          0.0      

## From Scratch - ID3

### 1. Tính entropy

In [ ]:
import numpy as np

def calculate_entropy(target):
  """
  Calculates the entropy of a target variable.

  Args:
    target: A pandas Series representing the target variable.

  Returns:
    The calculated entropy value.
  """
  class_counts = target.value_counts()
  probabilities = class_counts / len(target)
  entropy = -np.sum(probabilities * np.log2(probabilities + 1e-9)) # Add a small epsilon to avoid log(0)
  return entropy

# Calculate entropy for the 'play' variable
entropy_play = calculate_entropy(y)
print(f"Entropy of 'play' variable: {entropy_play:.4f}")

Entropy of 'play' variable: 0.9403


### 2. Tính information gain

In [ ]:
def calculate_information_gain(df, attribute_col, target_col):
  """
  Calculates the information gain for a given attribute.

  Args:
    df: A pandas DataFrame.
    attribute_col: The name of the attribute column.
    target_col: The name of the target variable column.

  Returns:
    The calculated information gain.
  """
  original_entropy = calculate_entropy(df[target_col])
  weighted_entropy = 0

  for value in df[attribute_col].unique():
    subset = df[df[attribute_col] == value]
    subset_entropy = calculate_entropy(subset[target_col])
    proportion = len(subset) / len(df)
    weighted_entropy += proportion * subset_entropy

  information_gain = original_entropy - weighted_entropy
  return information_gain

# Example usage: Calculate Information Gain for 'outlook'
ig_outlook = calculate_information_gain(df, 'outlook', 'play')
print(f"Information Gain for 'outlook': {ig_outlook:.4f}")

# Example usage: Calculate Information Gain for 'temperature'
ig_temperature = calculate_information_gain(df, 'temperature', 'play')
print(f"Information Gain for 'temperature': {ig_temperature:.4f}")

# Example usage: Calculate Information Gain for 'humidity'
ig_humidity = calculate_information_gain(df, 'humidity', 'play')
print(f"Information Gain for 'humidity': {ig_humidity:.4f}")

# Example usage: Calculate Information Gain for 'wind'
ig_wind = calculate_information_gain(df, 'wind', 'play')
print(f"Information Gain for 'wind': {ig_wind:.4f}")

Information Gain for 'outlook': 0.2467
Information Gain for 'temperature': 0.0292
Information Gain for 'humidity': 0.1518
Information Gain for 'wind': 0.0481


### Xây dựng cây quyết định bằng ID3

In [ ]:
def build_id3_tree(df, target_col, attributes):
    """
    Recursively builds an ID3 decision tree.

    Args:
        df: The current DataFrame subset.
        target_col: The name of the target variable column.
        attributes: A list of attribute columns to consider for splitting.

    Returns:
        A dictionary representing the decision tree node or a leaf node value.
    """
    # Base Case 1: If all instances in the subset belong to the same class
    if len(df[target_col].unique()) == 1:
        return df[target_col].iloc[0]

    # Base Case 2: If there are no more attributes to split on or the DataFrame is empty
    if not attributes or df.empty:
        # Return the majority class
        return df[target_col].mode()[0]

    # Recursive Step: Find the best attribute to split on
    information_gains = {}
    for attribute in attributes:
        information_gains[attribute] = calculate_information_gain(df, attribute, target_col)

    best_attribute = max(information_gains, key=information_gains.get)

    # Create the tree node
    tree = {best_attribute: {}}

    # Get remaining attributes for recursive calls
    remaining_attributes = [attr for attr in attributes if attr != best_attribute]

    # Build subtrees for each unique value of the best attribute
    for value in df[best_attribute].unique():
        subset = df[df[best_attribute] == value].copy()  # Use .copy() to avoid SettingWithCopyWarning
        subtree = build_id3_tree(subset, target_col, remaining_attributes)
        tree[best_attribute][value] = subtree

    return tree

# Define the attributes (excluding 'id' and 'play')
attributes = ['outlook', 'temperature', 'humidity', 'wind']

# Build the ID3 tree
id3_tree = build_id3_tree(df.copy(), 'play', attributes) # Use a copy to avoid modifying the original df

# Print the built tree (optional, for visualization)
import json
print("Built ID3 Tree:")
print(json.dumps(id3_tree, indent=2))

Built ID3 Tree:
{
  "outlook": {
    "sunny": {
      "humidity": {
        "high": "no",
        "normal": "yes"
      }
    },
    "overcast": "yes",
    "rainy": {
      "wind": {
        "weak": "yes",
        "strong": "no"
      }
    }
  }
}


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score

# 1. Tách tập dữ liệu df thành các tập huấn luyện và kiểm tra.
# Using X_processed (encoded features) and y (target) from previous steps
# This assumes X_processed and y are available from prior cells.
X_train_scratch, X_test_scratch, y_train_scratch, y_test_scratch = train_test_split(
    X_processed, y, test_size=0.2, random_state=42
)

# Combine X_train_scratch and y_train_scratch for building the tree
# The build_id3_tree function expects a single DataFrame
train_df_scratch = X_train_scratch.copy()
train_df_scratch[y.name] = y_train_scratch.values # Add the target column to the training DataFrame

# 2. Xây dựng cây ID3 trên tập dữ liệu huấn luyện
# We need the original column names for the build_id3_tree function
# Let's use the original categorical column names
original_attributes = ['outlook', 'temperature', 'humidity', 'wind']

# The build_id3_tree function expects a DataFrame with original categorical columns.
# We need to reverse the one-hot encoding for training the scratch model.
# Let's create a training DataFrame with original categorical columns.
# This requires re-using the original df and splitting it based on the train/test indices.

train_indices, test_indices = train_test_split(
    df.index, test_size=0.2, random_state=42
)

df_train_scratch = df.loc[train_indices].copy()
df_test_scratch = df.loc[test_indices].copy()

# Build the ID3 tree using the original categorical training data
id3_tree_scratch = build_id3_tree(df_train_scratch.drop('id', axis=1), 'play', original_attributes)


# 3. Dự đoán nhãn cho tập dữ liệu kiểm tra
predictions_scratch = []
# Iterate through each row of the test DataFrame (with original columns)
for index, row in df_test_scratch.drop('id', axis=1).iterrows():
    data_point = row.to_dict()
    prediction = predict_id3(id3_tree_scratch, data_point)
    predictions_scratch.append(prediction)

# 4. Chuyển danh sách các dự đoán thành một Series của pandas
y_pred_scratch = pd.Series(predictions_scratch, index=df_test_scratch.index)

# 5. Tính toán độ chính xác của mô hình
accuracy_scratch = accuracy_score(y_test_scratch, y_pred_scratch)

# 6. In độ chính xác.
print(f"Độ chính xác của mô hình ID3 từ scratch: {accuracy_scratch:.2f}")

Độ chính xác của mô hình ID3 từ scratch: 1.00


## Dùng Scikit Learn


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Khởi tạo và huấn luyện mô hình cây quyết định
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Độ chính xác của mô hình: {accuracy:.2f}")
print("\nBáo cáo phân loại:")
print(report)

Độ chính xác của mô hình: 1.00

Báo cáo phân loại:
              precision    recall  f1-score   support

          no       1.00      1.00      1.00         1
         yes       1.00      1.00      1.00         2

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

